<a href="https://colab.research.google.com/github/mlizander/chefbot/blob/main/ChefBot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#Instalando o SDK do Google
!pip install -q -U google-generativeai

In [14]:

#Imports de bibliotecas
import requests
import random
import google.generativeai as genai

#função que alimenta as listas de mensagens com o conteúdos dos JSON
def busca_dados_json(url):
  data = requests.get(url)
  return data.json()

# traz os dados da saudação do JSON
saudacoes = busca_dados_json('https://raw.githubusercontent.com/mlizander/chefbot/main/saudacao.json')
incentivos = busca_dados_json('https://raw.githubusercontent.com/mlizander/chefbot/main/incentivo.json')
despedidas_sem_receita = busca_dados_json('https://raw.githubusercontent.com/mlizander/chefbot/main/depedida_sem_receita.json')
despedidas_uma_receita = busca_dados_json('https://raw.githubusercontent.com/mlizander/chefbot/main/despedida_uma_receita.json')
despedidas_mais_receitas = busca_dados_json('https://raw.githubusercontent.com/mlizander/chefbot/main/despedida_mais_receitas.json')
texto_nao_mandou_lista = busca_dados_json('https://raw.githubusercontent.com/mlizander/chefbot/main/nao_mandou_lista.json')
buscando = busca_dados_json('https://raw.githubusercontent.com/mlizander/chefbot/main/buscando.json')

#inicia a variável de controle de quantas receitas esse chat já gerou
count_receitas = 0
model_name = "gemini-1.0-pro"
configurations = {
  "candidate_count": 1,
  "temperature": 1,
}

#inicia a API do Google Gemini
from google.colab import userdata
api_key = userdata.get('SECRET_KEY')
genai.configure(api_key=api_key)

chefbot = genai.GenerativeModel(model_name=model_name,
                                generation_config=configurations)

chat = chefbot.start_chat(history=[])

has_fail = 0
prompt = "##"

#roda o laço principal, até que o usuário informe sair no chat
while prompt.lower() != "sair":
  #verifica se houve alguma falha, por exemplo, o usuário digitou algo que não era uma lista, ou a lista informada não possui nenhum item de culinária
  if (has_fail == 0):
    #chama a mensagem de saudação (caso seja a primeira receita a ser pedida) ou de incentivo para o que o usuário obtenha mais receitas (caso já tenha obtido alguma receita nessa sessão)
    if (count_receitas == 0):
      print(random.choice(saudacoes)["saudacao"])
    else:
      print(random.choice(incentivos)["incentivo"])
  has_fail = 0
  #solicita os ingredientes (prompt)
  prompt = input('Insira somente os ingredientes ou digite sair para encerrar: ')
  if (prompt.lower() != "sair"):
    #envia o prompt para o Google Gemini, perguntando ao mesmo se o que foi digitado se trata de uma lista
    message = f'considere o texto a seguir: {prompt}.\n\n Responda apenas com sim ou não: o texto informado é uma lista?'
    response = chat.send_message(message)
    if (response.text.lower() == "sim"):
        #Pergunta ao Google Gemini se a lista contém ingredientes de culinária, mesmo que tenha itens que não sejam de culinárias (tratamento de discrepâncias)
        message = 'Responda apenas com sim ou não: a lista enviada possui ingredientes de culinária?'
        possui_itens = chat.send_message(message)
        if (possui_itens.text.lower() == "sim"):
          #Uma vez que tem ingredientes de culinária, solicita ao Google Gemini uma receita que use os ingredientes, ainda que não use todos
          message = 'Aja, então, como um chef de cozinha e considere que esses são os ingredientes que possuo. Crie uma receita que eu possa fazer com esses ingredientes. Não há a necessidade de se usar todos os ingredientes da lista.'
          receita = chat.send_message(message)
          print('----------------------------------------')
          #Envia uma mensagem na tela como se o chef estivesse buscando a receita e encontrando
          print(random.choice(buscando)["resposta"])
          print('----------------------------------------')
          #exibe a receita encontrada
          print(receita.text, '\n\n')
          print('----------------------------------------')
          count_receitas = count_receitas + 1
        else:
          #exibe um texto de que o usuário não mandou uma lista ou mandou uma lista de itens que não são ingredientes de culinária
          print(random.choice(texto_nao_mandou_lista)["resposta"])
          has_fail = 1
    else:
      #exibe um texto de que o usuário não mandou uma lista ou mandou uma lista de itens que não são ingredientes de culinária
      print(random.choice(texto_nao_mandou_lista)["resposta"])
      has_fail = 1

#Encerramento. O usuário informou sair. Exibe mensagem de acordo com o que o usuário obteve. Mensagens diferentes para quem não obteve nenhuma receita, uma ou mais de uma.
print('----------------------------------------')
if (count_receitas == 0):
  print(random.choice(despedidas_sem_receita)["despedida"])
else:
  if (count_receitas == 1):
    print(random.choice(despedidas_uma_receita)["despedida"])
  else:
    print(random.choice(despedidas_mais_receitas)["despedida"])



Bem-vindo, chef! 👨‍🍳 Diga-me o que você tem na despensa e vamos criar algo delicioso.
Insira somente os ingredientes ou digite sair para encerrar: limão, farinha de trigo, fermento, leite 
----------------------------------------
Deixe-me pensar... 🤔 Já sei! Essa receita vai te surpreender!
----------------------------------------
**Pão de Limão Caseiro**

**Ingredientes:**

* 1 xícara de farinha de trigo
* 1 colher de chá de fermento em pó
* 1/2 colher de chá de sal
* 1/4 xícara de açúcar
* 1 ovo
* 1/2 xícara de leite
* Raspas de 1 limão

**Instruções:**

1. Pré-aqueça o forno a 180°C. Unte e enfarinhe uma forma de pão de 23x13 cm.
2. Em uma tigela média, misture a farinha, o fermento, o sal e o açúcar.
3. Em uma tigela separada, misture o ovo, o leite e as raspas de limão.
4. Adicione os ingredientes úmidos aos ingredientes secos e misture até formar uma massa macia.
5. Despeje a massa na forma de pão preparada e asse por 25-30 minutos, ou até que um palito inserido no centro saia li